In [58]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

""" Import Statements """

# Classics
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

# Import Statements
from sklearn.pipeline import Pipeline

from sklearn.linear_model import SGDClassifier#stochastic gradient descent
from sklearn.feature_extraction.text import TfidfVectorizer

# Experiment Management
from sklearn.model_selection import GridSearchCV

from sklearn.decomposition import TruncatedSVD

import spacy
nlp = spacy.load("en_core_web_md")

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [20]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [21]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [22]:
yelp.shape

(10000, 9)

In [23]:
def tokenize(doc):
    return [token for token in simple_preprocess(doc)]

yelp['tokens'] = yelp['text'].apply(lambda x: tokenize(x))
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,tokens
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w,"[beware, fake, fake, fake, we, also, own, smal..."
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA,"[came, here, for, lunch, togo, service, was, q..."
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA,"[ve, been, to, vegas, dozens, of, times, and, ..."
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ,"[we, went, here, on, night, where, they, close..."
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA,"[to, stars, not, bad, for, the, price, for, lu..."


## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

# create the transformer
vectorizer = CountVectorizer(stop_words='english')

# tokenize and build vocab
vectorizer.fit(yelp['text'])


# Create a Vocabulary
# The vocabulary establishes all of the possible words that we might use.
vectorizer.vocabulary_

# The vocabulary dictionary does not represent the counts of words!!
dtm = vectorizer.transform(yelp['text'])

In [25]:
# Get Word Counts for each document
dtmdf = pd.DataFrame(dtm.todense(), columns = vectorizer.get_feature_names())
dtmdf.head()

,00,000,001695,007,00a,00am,00ish,00pm,01,011802,02,02022015,02pm,03,04,05,05pm,06,07,08,09,0d,0mph,10,100,1000,1001,100k,100lb,100s,100th,100x,101,102,1026,1030am,104,105,106,106lb,107,108,109,1096,1099,10am,10k,10min,10mins,10news,10p,10pm,10th,10times,10x,10years,10yrs,11,110,110th,111,1115a,1130,115,117,119,11a,11am,11ish,11pm,12,120,1200,1221,123,123lbs,1245p,125,1260,129,12am,12k,12months,12oz,12pk,12pm,12th,12か所のコーナーのステアリングさばきは実にお見事でとてもいい体験になりました,13,130,131,135,139,13th,14,140,1400,14000,141,144,1440,145,149,14oz,14th,15,150,1500,150k,155,15am,15ish,15k,15min,15mins,15pm,15th,15をくぐる形,15を南へ15,16,160,1611,165,167,168,169,16oz,16th,17,170,1700,175,176,17th,18,180,1800,182,1830,189,1893,18and,18pretty,19,190,1900,1910s,1920,195,1950,1956,1960,1965,1969,196ドルから体験できる憧れのスーパーカーのコクピット,1970,1970s,1971,1975,1976,198,1980,1980s,1982,1984,1990,1990s,1993,1995,1996,1998,1999,19c,19h,19th,1am,1er,1hr,1k,1lb,1pm,1st,1weeks,1x1,1yr,20,200,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,200gm,2010,2011,2012,2013,2014,2015,2016,2017,2018,202,203,2037,205,2079,20am,20f,20k,20lbs,20min,20mins,20pc,20pm,20pp,20s,20th,20分程度,21,210,214,215,217,218,21million,21st,22,2200,229,22nd,22oz,23,230,230pm,231,234,235,23rd,23th,24,240,2400,241,246000,2495,24c,24hour,24hours,24hr,24hrs,24oz,24pk,24th,24x7,25,250,2500,254,25bucks,25c,25min,25minutes,25pm,25th,25x8,26,260,2625,26th,27,270,2700,273,275,279,27pm,27th,28,280,2800,28226,28319,28th,29,290,2900,292,2955650,2961,299,2am,2bath,2bed,2cats,2d,2days,2djs,2ea,2ft,2hours,2hr,2hrs,2k,2lbs,2meat,2milesandmore,2min,2minute,2nd,2oz,2pm,2v29rpxqqnawvxqlltyqpq,2x,2x1x1,2x2,2x6,2years,2yr,2yrs,30,300,3000,3000km,300g,301,30a,30am,30ish,30k,30m,30min,30mins,30ml,30oz,30p,30pm,30s,30th,31,310,315,319,32,320,324,32nd,32st,33,330,330ci,34,345,345pm,35,350,35ft,35lbs,35pm,35th,36,360,365,36th,37,370z,375,38,383,39,39th,3am,3d,3hours,3k,3oz,3p,3pm,3rd,3s,3x,3xcd,3yr,3yrs,40,400,4000,400sqft,400th,401,405,40ft,40mins,40mph,40pm,40s,40th,41,410,411,416,417,41716,42,420,4200,429,43,430,44,440,4402,45,450,45000,451,458,45min,45mins,45mns,45p,45pm,46,464,47,48,480,4850,49,490,495,499,49ers,49th,4am,4g,4hrs,4k,4lbs,4mins,4oz,4play,4pm,4runner,4s,4star,4th,4x,50,500,5000,50704,50am,50cents,50percent,50pm,50s,50th,51,512,51st,52,520,521,53,530,531,533,53rd,54,540,540i,54th,55,550,5500,550ならフルスロットルパッケージ,550ガヤルドを体験,551,5592,55am,55pm,56,563,57,570,5701,5751,58,58pm,59,596,599,5am,...,wss,wt,wtf,wth,wurst,wwa,wwe,wwii,www,wychwood,wyck,wylie,wyman,wynn,wyoming,x3,x5,xa,xanax,xavier,xb,xcite,xd,xerox,xi,xiao,xinjiang,xiu,xl,xlb,xmas,xo,xoxo,xp,xpel,xrays,xs,xtra,xx,y4,ya,yaaaaaasss,yacht,yada,yag,yah,yahoo,yainni,yakatori,yaki,yakisoba,yakitori,yall,yam,yama,yams,yang,yangs,yankees,yanking,yankovic,yapping,yard,yardage,yardbird,yards,yarn,yasmin,yawn,yay,yayyy,yaz,yc,ycs,yea,yeaaaaaa,yeah,yeahhhhhhh,yeahs,year,yearly,years,yeast,yeasted,yeay,yeayy,yeeps,yekp,yell,yelled,yelling,yellow,yellowed,yellowtail,yells,yelp,yelpaholics,yelped,yelper,yelpers,yelpgta,yelping,yelposphere,yep,yes,yesnick,yessss,yesterdat,yesterday,yesterdays,yesteryear,yeti,yeux,yg,yhe,yield,yik,yikes,yin,ying,yinzer,yiu,yknow,ymca,ymmm,ynot,yo,yoga,yoghurt,yogi,yogourt,yogurt,yogurtland,yogurts,yokes,yolk,yolks,yolky,yolo,yonge,yoo,yoonha,yore,york,yorkdale,yorker,yorkers,yorkie,yorkshire,yorkville,yoself,youll,young,younger,youngest,youngins,younglady,youngster,youngstown,youngtown,yountville,youre,yousaku,youth,youthful,youtube,yovi,yowza,yowzaz,yqpzawu4du69tv4kqna6xq,yr,yrs,ytz,yu,yuba,yuca,yucatan,yuck,yuckie,yucky,yuengling,yuk,yukon,yul,yum,yumm,yummers,yummi,yummie,yummiest,yumminess,yummm,yummmm,yummmmmm,yummmmms,yummmmmy,yummmmo,yummms,yummmy,yummo,yummy,yumyumyum,yung,yup,yupha,yuppie,yuppy,yuzu,yvette,yyz,za,zaba,zabas,zac,zach,zack,zagat,zales,zambroski,zane,zanet,zanzibar,zap,zara,zaru,zealand,zebra,zedd,zen,zeppe,zerdin,zero,zerorez,zest,zesto,zesty,zeus,zgv0zroif956gw4cul8mha,zha,zhou,ziggy,ziggys,zillion,zimbabwe,zimbawean,zimbrick,zinbu

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

# Create a vocabulary and get word counts per document
dtm = tfidf.fit_transform(yelp['text'])
#dtm = tfidf.transform(new_data)

# Print word counts

# Get feature names to use as dataframe column headers

# View Feature Matrix as DataFrame
docs = pd.DataFrame(dtm.todense(), columns = tfidf.get_feature_names())
docs.head()

,00,000,00pm,07,10,100,1000,101,10pm,11,110,11am,11pm,12,120,13,14,15,150,16,17,18,19,1pm,1st,20,200,2006,2011,2012,2013,2014,2015,2016,2017,2018,21,22,23,24,25,250,26,27,28,29,2nd,2pm,30,300,30am,30pm,35,39,3pm,3rd,40,400,45,48,49,4pm,4th,50,500,55,5pm,60,600,65,6pm,70,75,7pm,7th,80,88,8pm,90,95,99,9pm,ability,able,absolute,absolutely,ac,accent,accept,acceptable,accepted,access,accessible,accessories,accident,accidentally,accommodate,accommodated,accommodating,accomodating,accompanied,according,account,accurate,acknowledge,acknowledged,acne,acrylic,act,acted,action,active,activities,activity,acts,actual,actually,ad,adam,add,added,addicted,adding,addition,additional,additionally,address,addressed,adds,adequate,adjacent,adjust,adjustment,admission,admit,admitted,adorable,adult,adults,advance,advantage,adventure,adventurous,advertise,advertised,advice,advise,advised,afford,affordable,afraid,afternoon,agave,age,aged,agent,ages,aggressive,ago,agree,agreed,ah,ahead,ahi,ai,aid,ain,aioli,air,airline,airlines,airport,airports,airy,aka,al,alas,alcohol,alcoholic,ale,alex,alfredo,alignment,alike,allergic,allergies,allergy,alley,allow,allowed,allows,almond,alot,alright,alternative,alternatives,amanda,amazed,amazing,amazingly,amazon,ambiance,ambience,amenities,america,american,ami,amounts,ample,amy,andrew,angel,angela,angeles,angry,animal,animals,anniversary,annoyed,annoying,answer,answered,answering,answers,anthony,anxious,anybody,anymore,anytime,anyways,apart,apartment,apartments,apologetic,apologies,apologize,apologized,apologizing,apology,app,apparently,appeal,appealing,appear,appearance,appeared,appears,appetite,appetizer,appetizers,appetizing,apple,applebee,apples,appliances,application,applied,apply,appointment,appointments,appreciate,appreciated,approach,approached,appropriate,approximately,apps,appt,april,aquarium,area,areas,aren,argue,arguing,aria,arizona,arm,arms,arranged,arrangement,array,arrival,arrive,arrived,arriving,arrogant,arrowhead,art,artichoke,artisan,artist,artists,arts,arugula,asada,asap,ashley,asia,asian,aside,ask,asked,asking,asks,asleep,asparagus,aspects,ass,assist,assistance,assistant,associate,assortment,assume,assumed,assuming,assure,assured,asu,ate,atmosphere,atop,attached,attack,attempt,attempted,attempting,attempts,attend,attendant,attended,attending,attention,attentive,attitude,attitudes,attorney,attractive,au,audience,august,aussi,austin,authentic,authenticity,auto,automatic,automatically,availability,available,ave,avec,avenue,average,avid,avocado,avoid,avoided,aware,away,awe,awesome,awful,awhile,awkward,ayce,az,baby,bachelorette,background,backyard,bacon,bad,badly,bag,bagel,bagels,bags,baguette,baked,bakery,baklava,balance,balanced,balcony,ball,balloons,balls,balsamic,banana,bananas,band,bands,bang,bank,bar,barbecue,barbeque,barber,barbers,barbershop,bare,barely,bargain,barista,baristas,bars,bartender,bartenders,base,baseball,based,basement,basic,basically,basics,basil,basis,basket,basketball,bass,bat,batch,bath,bathroom,bathrooms,baths,batter,battered,battery,bay,bbb,bbq,bc,bday,beach,bean,beans,bear,beat,beats,beautiful,beautifully,beauty,bed,bedroom,beds,beef,beer,beers,beet,beets,began,begin,beginning,behavior,behold,believe,bell,bellagio,belle,belly,belt,ben,benches,benedict,benefit,benefits,benny,bento,berries,berry,best,bet,better,beverage,beverages,beware,bf,bianco,big,bigger,biggest,biggie,bike,bikes,billing,bills,bird,birds,birthday,biryani,biscuit,biscuits,bison,bisque,bistro,bit,...,thorough,thoroughly,thought,thoughtful,thoughts,thousands,threw,thrilled,throw,throwing,thrown,thumbs,thursday,ticket,tickets,tiger,tight,tikka,til,tile,till,tim,time,timely,timers,times,timing,tint,tiny,tip,tipped,tips,tiramisu,tire,tired,tires,tissue,toast,toasted,today,toddler,toe,toes,toffee,tofu,toilet,told,tom,tomato,tomatoes,tomorrow,ton,tone,tongue,tonic,tonight,tons,tony,took,tools,tooth,topped,topping,toppings,tops,torn,toronto,tortilla,tortillas,tossed,total,totally,tots,touch,touched,touches,tough,tour,

In [44]:
# Instantiate
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')

# Fit on TF-IDF Vectors
nn.fit(dtm.todense())

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [45]:
# Query Using kneighbors 
nn.kneighbors(dtm.todense()[0])

(array([[0.        , 1.        , 1.        , 1.25325603, 1.25954602]]),
 array([[   0, 6311, 6204, 2131, 3276]], dtype=int64))

In [46]:
yelp.sample(2)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,tokens
711,_cjPEH9wXhKS-HQe_U3M4Q,0,2015-07-20 01:35:07,0,ukOt3ry_xXt6YmPqJA86sg,5,"I went for the first time today. Wow, the fre...",0,7GS60aGKil36XCO4vLGDgQ,"[went, for, the, first, time, today, wow, the,..."
4297,e13SEvJud_vgeDR_doL4sQ,0,2013-07-29 21:07:28,0,uwoysbaEnQBABnnSVzi0eg,4,Stopped here for a QUICK bite before our show....,1,QraRy2Y1RYSRbH7WJN5_cw,"[stopped, here, for, quick, bite, before, our,..."


In [47]:
yelp['text'].iloc[8339]

"I really enjoy the spiciness of the Arizona burger.\n\nWith today's meal we got a little confused and mis-ordered something but Jalissa, the manager, was more than eager to fix it for us.  There was a lot of positive energy from the staff and they kept things moving smoothly.\n\nI'm impressed with both the food and, especially, the service.  My biggest problem is being careful to not over-order and get the shake and the fries and the everything else.\n\nOverall, I've been very pleased with my meals at this location."

In [48]:
#tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

fake_review = ['''I thought the service was excellent, I really liked the food. I wish the price was more reasonable, but overall the experience was good.''']
new = tfidf.transform(fake_review)
nn.kneighbors(new.todense())

(array([[1.        , 1.        , 1.15384855, 1.17164522, 1.18575431]]),
 array([[6204, 6311, 3693, 1781, 3062]], dtype=int64))

In [53]:
yelp['text'].iloc[1781]

'Excellent place to go to. Our waitress Laura was great in taking care of me and my friend. The food overall is pretty good and not expensive. We loved our overall experience and we hope to be back soon.'

In [52]:
yelp['text'].iloc[3062]

'Excellent place!, excellent service, good food, expect to pay about 20.00 for a burger but its an experience and its vegas!!'

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [55]:
# Create Pipeline

vect = TfidfVectorizer(stop_words='english')#instantiate vectorizer
sgdc = SGDClassifier()#instantiate classifier

pipe = Pipeline([('vect', vect), ('clf', sgdc)])#put objects in pipe#the last item in pipeline, should have fit command
#vectorizor, classifier

# Fit Pipeline
#pipe.fit(data.data, data.target)
pipe.fit(yelp['text'], yelp['stars'])

C:\Users\lilyx\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))])

In [56]:
pipe.predict(['I thought the service was excellent, I really liked the food. I wish the price was more reasonable, but overall the experience was good.'])

array([4], dtype=int64)

In [59]:
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),#word appears than more than 50% of documents, excluded, 
    #max df is a parameter of tfidf
    #vect__min_df works similarly must be inside at least 2% 
    #'vect__min_df': (.02, .05),
    'vect__max_features':(500, 500, 1000),#total number of words tfidf will learn after above parameters
    'clf__max_iter':(2, 10, 10)#number of times makes a pass search 
    #max_iter SGDClassifier
}
grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(yelp['text'], yelp['stars'])

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   21.0s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:  1.1min finished
C:\Users\lilyx\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.5, 0.75, 1.0), 'vect__max_features': (500, 500, 1000), 'clf__max_iter': (2, 10, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [60]:
pipe.predict(['I thought the service was excellent, I really liked the food. I wish the price was more reasonable, but overall the experience was good.'])

array([4], dtype=int64)

#### Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)

In [118]:
import spacy#how to do everything above using spacy
from spacy.tokenizer import Tokenizer

nlp = spacy.load("en_core_web_md")
# Tokenizer
tokenizer = Tokenizer(nlp.vocab)
tokens = []

""" Update those tokens w/o stopwords"""
for doc in tokenizer.pipe(yelp['text'], batch_size=500):
    doc_tokens = []
    
    for token in doc:
        if (token.is_stop == False) & (token.is_punct == False):
            doc_tokens.append(token.text.lower())
    tokens.append(doc_tokens)
            
yelp['tokens2'] = tokens

In [119]:
# A Dictionary Representation of all the words in our corpus
id2word = corpora.Dictionary(yelp['tokens2'])#fit portion of vectorizor
corpus = [id2word.doc2bow(text) for text in yelp['tokens2']]

In [120]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,#for reproducability not very stable, different results
                   num_topics = 15,
                   passes=10,
                   workers=4#multi-core
                  )

lda.print_topics()#topics are list of tuples, id of topic, and probability * word

[(0,
  '0.019*"\n\n" + 0.016*" " + 0.006*"good" + 0.006*"like" + 0.004*"place" + 0.004*"got" + 0.003*"time" + 0.003*"don\'t" + 0.003*"didn\'t" + 0.002*"told"'),
 (1,
  '0.023*"\n\n" + 0.008*"food" + 0.007*"good" + 0.007*" " + 0.007*"chicken" + 0.007*"like" + 0.006*"great" + 0.005*"place" + 0.005*"little" + 0.004*"\n"'),
 (2,
  '0.021*" " + 0.018*"\n\n" + 0.007*"place" + 0.006*"food" + 0.006*"\n" + 0.005*"it\'s" + 0.004*"good" + 0.003*"told" + 0.003*"service" + 0.003*"didn\'t"'),
 (3,
  '0.028*"\n\n" + 0.006*"it\'s" + 0.005*"like" + 0.005*"place" + 0.004*"good" + 0.004*" " + 0.004*"people" + 0.004*"got" + 0.004*"food" + 0.003*"went"'),
 (4,
  '0.011*"\n" + 0.007*"time" + 0.007*"\n\n" + 0.006*"service" + 0.006*"place" + 0.005*"food" + 0.005*"good" + 0.005*"best" + 0.004*"like" + 0.004*"i\'ve"'),
 (5,
  '0.068*" " + 0.009*"\n\n" + 0.008*"\n" + 0.006*"good" + 0.005*"food" + 0.005*"like" + 0.005*"time" + 0.005*"great" + 0.004*"place" + 0.004*"service"'),
 (6,
  '0.026*"\n" + 0.007*"\n\n" + 

In [121]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]#cleaning
topics = [' '.join(t[0:5]) for t in words]
for t in topics:
    print(t)
    print('\n')
#print(topics[0])#typically use the first 3-5 words for visualization purposes



   good like place




 food good   chicken


  

 place food 





 it's like place good



 time 

 service place


  

 
 good food



 

 place got it's




   de good la




 great place food like


  

 
 good service


  

 food place service




 like don't i've time


food best service place love


  

 great place like




 great place food best




In [ ]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, id2word)